In [10]:
import pandas as pd
import glob

In [153]:
train_path = glob.glob('./健康成人脑龄预测挑战赛公开数据/Train/*.csv')
train_path.sort()
train_set = pd.concat([
    pd.read_csv(x).sort_values(by='subject_ID').drop('subject_ID', axis=1)
    for x in train_path
], axis=1)
train_set = train_set.drop('注：性别（1=Female, 2=Male)', axis=1)

test_path = glob.glob('./健康成人脑龄预测挑战赛公开数据/Test/*.csv')
test_path.sort()
test_set = pd.concat([
    pd.read_csv(x, encoding='latin-1').sort_values(by='subject_ID').drop('subject_ID', axis=1)
    for x in test_path
], axis=1)
test_set = test_set.rename({'MRIÉ¨ÃèÒÇÀàÐÍ': 'MRI扫描仪类型'}, axis=1)
test_set = test_set.rename({'ÐÔ±ð': '性别'}, axis=1)
test_set = test_set.rename({'×¢£ºÐÔ±ð£¨1=Female, 2=Male)': '注：性别（1=Female, 2=Male)'}, axis=1)
test_set = test_set.drop('注：性别（1=Female, 2=Male)', axis=1)
test_set = test_set.iloc[:389]

train_set = train_set.reset_index(drop=True)
test_set = test_set.reset_index(drop=True)

train_set.shape, test_set.shape

((1600, 410), (389, 409))

In [154]:
pd.read_csv('./健康成人脑龄预测挑战赛公开数据/Train/subject_info - 1600.csv').head()

,subject_ID,MRI扫描仪类型,性别,年龄,"注：性别（1=Female, 2=Male)"
0,CNBD_00002,GE,2,25,NaN
1,CNBD_00003,GE,1,26,NaN
2,CNBD_00007,SIEMENS,2,27,NaN
3,CNBD_00008,SIEMENS,2,32,NaN
4,CNBD_00009,SIEMENS,2,32,NaN


In [161]:
submit = pd.read_csv('提交示例.csv')

In [155]:
train_set.corr()

,Left-Cerebellum-White-Matter,Left-Cerebellum-Cortex,Left-Thalamus,Left-Caudate,Left-Putamen,Left-Pallidum,Brain-Stem,Left-Hippocampus,Left-Amygdala,Left-Accumbens-area,...,wm-rh-superiorfrontal,wm-rh-superiorparietal,wm-rh-superiortemporal,wm-rh-supramarginal,wm-rh-frontalpole,wm-rh-temporalpole,wm-rh-transversetemporal,wm-rh-insula,Left-UnsegmentedWhiteMatter,Right-UnsegmentedWhiteMatter
Left-Cerebellum-White-Matter,1.000000,0.378677,0.457433,0.149552,0.103914,0.399500,0.438675,0.361754,0.299071,-0.068085,...,0.275032,0.296434,0.310295,0.240172,0.227825,0.117091,0.019199,0.058093,0.282931,0.289592
Left-Cerebellum-Cortex,0.378677,1.000000,0.430878,0.336666,0.362759,0.406794,0.686377,0.416519,0.350246,0.181498,...,0.292680,0.317289,0.335298,0.278121,0.143805,0.063327,-0.047831,-0.057911,0.113572,0.167182
Left-Thalamus,0.457433,0.430878,1.000000,0.327967,0.325875,0.535907,0.496566,0.616459,0.488400,0.239467,...,0.432908,0.355724,0.416404,0.317875,0.244961,0.149244,0.004778,0.056766,0.334639,0.368616
Left-Caudate,0.149552,0.336666,0.327967,1.000000,0.585147,0.432192,0.373843,0.311065,0.239914,0.202209,...,0.340885,0.290348,0.326190,0.271395,0.092871,0.037698,-0.076941,-0.051758,0.123712,0.164032
Left-Putamen,0.103914,0.362759,0.325875,0.585147,1.000000,0.513188,0.375872,0.324406,0.348544,0.337787,...,0.345129,0.319689,0.326597,0.256712,0.114335,0.067698,-0.052365,-0.028891,0.159162,0.201776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wm-rh-temporalpole,0.117091,0.063327,0.149244,0.037698,0.067698,0.132729,0.118083,0.140736,0.168694,0.028487,...,0.073291,0.111848,0.068734,0.004839,0.165618,1.000000,0.884574,0.596202,0.192600,0.212964
wm-rh-transversetemporal,0.019199,-0.047831,0.004778,-0.076941,-0.052365,-0.023556,-0.043391,-0.038065,-0.016349,-0.037687,...,-0.142082,-0.074019,-0.149023,-0.164422,0.016811,0.884574,1.000000,0.689865,0.064045,0.039294
wm-rh-insula,0.058093,-0.057911,0.056766,-0.051758,-0.028891,0.014695,-0.001540,0.027236,0.040880,-0.030037,...,-0.004503,-0.035574,0.083113,-0.152628,0.149148,0.596202,0.689865,1.000000,0.413231,0.129671
Left-UnsegmentedWhiteMatter,0.282931,0.113572,0.334639,0.123712,0.159162,0.316553,0.294262,0.269819,0.207477,0.056382,...,0.465004,0.421049,0.478957,0.291255,0.480735,0.192600,0.064045,0.413231,1.000000,0.907823


In [156]:
train_set.shape, test_set.shape

((1600, 410), (389, 409))

In [157]:
train_set['MRI扫描仪类型'] = train_set['MRI扫描仪类型'].map({'GE': 1, 'SIEMENS': 0})
test_set['MRI扫描仪类型'] = test_set['MRI扫描仪类型'].map({'GE': 1, 'SIEMENS': 0})

In [165]:
from sklearn.tree import DecisionTreeRegressor
m = DecisionTreeRegressor()
m.fit(
    train_set.drop('年龄', axis=1),
    train_set['年龄']
)

submit['年龄'] = m.predict(test_set)
submit.to_csv('dt.csv', index=None)

In [169]:
from lightgbm import LGBMRegressor
m = LGBMRegressor()
m.fit(
    train_set.drop('年龄', axis=1).values,
    train_set['年龄']
)
submit['年龄'] = m.predict(test_set.values)


from catboost import CatBoostRegressor
m = CatBoostRegressor()
m.fit(
    train_set.drop('年龄', axis=1).values,
    train_set['年龄']
)

submit['年龄'] += m.predict(test_set.values)


from xgboost import XGBRegressor
m = XGBRegressor()
m.fit(
    train_set.drop('年龄', axis=1).values,
    train_set['年龄']
)

submit['年龄'] += m.predict(test_set.values)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007260 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 91770
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 409
[LightGBM] [Info] Start training from score 43.359375
Learning rate set to 0.044099
0:	learn: 13.4818751	total: 73.1ms	remaining: 1m 12s
1:	learn: 13.3774536	total: 86.8ms	remaining: 43.3s
2:	learn: 13.2949890	total: 100ms	remaining: 33.4s
3:	learn: 13.2049775	total: 115ms	remaining: 28.6s
4:	learn: 13.1243518	total: 128ms	remaining: 25.5s
5:	learn: 13.0529386	total: 142ms	remaining: 23.4s
6:	learn: 12.9800796	total: 155ms	remaining: 22s
7:	learn: 12.8935615	total: 169ms	remaining: 21s
8:	learn: 12.7991756	total: 183ms	remaining: 20.1s
9:	learn: 12.7106440	total: 196ms	remaining: 19.4s
10:	learn: 12.6414328	total: 210ms	remaining: 18.9s
11:	learn: 12.5808129	total: 224ms	remaining: 18.4s
12:	learn:

In [170]:
submit['年龄'] /= 3
submit.to_csv('3.csv', index=None)